In [7]:
import collections

import networkx as nx
%matplotlib inline
import matplotlib.pyplot as plt

import parser
import imp; imp.reload(parser)

<module 'parser' from '/Users/ianarawjo/Desktop/lambda_calc/analysis/parser.py'>

In [8]:
# Unfortunately, we have to do this manually, since the condition is always logged as 'A' for some reason.

events1, level_sequence1 = parser.read_events("userlogs/p1")
events2, level_sequence2 = parser.read_events("userlogs/p2")
events3, level_sequence3 = parser.read_events("userlogs/p3")
events4, level_sequence4 = parser.read_events("userlogs/p4")
events5, level_sequence5 = parser.read_events("userlogs/p5")
events6, level_sequence6 = parser.read_events("userlogs/p6")
events7, level_sequence7 = parser.read_events("userlogs/p7")

blocks_only = level_sequence3 + level_sequence6
verbatim = level_sequence4 + level_sequence7
constructive = level_sequence2 + level_sequence5 + level_sequence1

### Vis.js graph visualization
Note: you *must* add vis.js to your static paths in your jupyter notebook config file. Please see [this tutorial](https://www.codementor.io/isaib.cicourel/visjs-visualization-in-jupyter-notebook-phgb3fjv0) for details.

*(Eventually we will wrap this module in a .js file and also add it to static paths, but for now the logic is not finished yet.)*

In [9]:
# Convert some graph data into a JSON string
from networkx.readwrite import json_graph
import json
graph_to_json = lambda G: json.dumps(json_graph.node_link_data(G))
# test_data = graph_to_json(graphs[0])

merged = parser.mark_liveness(parser.get_complete_merged_graph(constructive, 25))
test_data = graph_to_json(merged)

# Pass graph data to JS backend
from IPython.display import Javascript
js = "window.graph_data={};".format(test_data)

# DO NOT ADD a ; to this line!! For some reason, it will not run it.
Javascript(js)

<IPython.core.display.Javascript object>

In [10]:
js = ["window.conditions = {};"]
for condition, levels in [("blocks", blocks_only), ("verbatim", verbatim), ("constructive", constructive)]:
    js.append("window.conditions['{}'] = [];".format(condition))
    
    for level_idx in range(67):
        merged = parser.mark_liveness(parser.get_complete_merged_graph(levels, level_idx))
        js.append("window.conditions['{}'].push({});".format(
            condition,
            graph_to_json(merged)
        ))
        
Javascript('\n'.join(js))

<IPython.core.display.Javascript object>

In [11]:
%%javascript

/* Converts the networkx output (serialized through JSON) 
*  to vis.js format. */
function nx_to_vis(G) {
    
    // Convert nodes from nx format into vis.js format:
    var nodes = G["nodes"].map(function(n, idx) {
        var background = "#D2E5FF";
        if (n["terminal"] && n["live"]) {
            background = "#7fc97f";
        }
        else if (n["terminal"]) {
            background = "red";
        }
        else if (n["initial"]) {
            background = "#beaed4";
        }
        else if (!n["live"]) {
            background = "orange";
        }
        return { 
            id: idx, 
            label: n["id"], 
            color: {
                background: background,
            }
        };
    });
    
    // Convert edges ('links') to vis.js format:
    var edges = G["links"].map(function(e) {
        var r = { from: e["source"], to: e["target"] };
        if ("count" in e) r.label = "" + e.count;
        if ("weight" in e) {
            if (e.weight > 0.75) {
                r.width = 3;
            }
            else if (e.weight > 0.25) {
                r.width = 1;
            }
            else {
                r.width = 0.5;
            }
        }
        if ("live" in e) {
            if (!e.live) {
                r.color = "red";
            }
        }
        return r;
    });
    
    return {
        nodes: nodes,
        edges: edges
    };
}

require.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis){
    
    // (a weird idempotency thing)
    $("#net1").remove();
    
    // Create an element to put the network into
    element.append("<div id='net1' style='border:solid 1px'></div>");
    
    // Load the JSON graph we sent from Python.
    var graph_json = window.graph_data;
    
    // Decode it to vis.js format:
    var data = nx_to_vis(graph_json);

    // Create the network
    var container = document.getElementById('net1');
    var options = {
        width: '800px',
        height: '800px',
        nodes: {
            shape: 'box'
        },
        edges: {
            arrows: 'to'
        }
    };
    var network = new vis.Network(container, data, options);
});

<IPython.core.display.Javascript object>

In [12]:
%%javascript

/* Converts the networkx output (serialized through JSON) 
*  to vis.js format. */
function nx_to_vis(G) {
    
    // Convert nodes from nx format into vis.js format:
    var nodes = G["nodes"].map(function(n, idx) {
        var background = "#D2E5FF";
        if (n["terminal"] && n["live"]) {
            background = "#7fc97f";
        }
        else if (n["terminal"]) {
            background = "red";
        }
        else if (n["initial"]) {
            background = "#beaed4";
        }
        else if (!n["live"]) {
            background = "orange";
        }
        return { 
            id: idx, 
            label: n["id"], 
            color: {
                background: background,
            }
        };
    });
    
    // Convert edges ('links') to vis.js format:
    var edges = G["links"].map(function(e) {
        var r = { from: e["source"], to: e["target"] };
        if ("count" in e) r.label = "" + e.count;
        if ("weight" in e) {
            if (e.weight > 0.75) {
                r.width = 3;
            }
            else if (e.weight > 0.25) {
                r.width = 1;
            }
            else {
                r.width = 0.5;
            }
        }
        if ("live" in e) {
            if (!e.live) {
                r.color = "red";
            }
        }
        return r;
    });
    
    return {
        nodes: nodes,
        edges: edges
    };
}

require.config({
    paths: {
        vis: 'vis'
    }
});

require(['vis'], function(vis) {
    var refresh = function() {
        var graph_json = window.conditions[cond.val()][parseInt(level.val(), 10)];
        
        if ($("#display").length > 0) {
        // (a weird idempotency thing)
            $("#display").empty();
        }
        else {
        // Create an element to put the network into
            element.append("<div id='display' style='border:solid 1px'></div>");
        }

        // Decode it to vis.js format:
        var data = nx_to_vis(graph_json);

        // Create the network
        var container = document.getElementById('display');
        var options = {
            width: '800px',
            height: '800px',
            nodes: {
                shape: 'box'
            },
            edges: {
                arrows: 'to'
            }
        };
        var network = new vis.Network(container, data, options);
    };
    
    var cond = $("<select id='cond'></select>");
    var level = $("<select id='lvl'></select>");
    for (var condName in window.conditions) {
        cond.append($("<option>" + condName + "</option>"));
    }
    
    var changeCond = function() {
        var current = parseInt(level.val(), 10);
        level.empty();
        for (var i = 0; i < window.conditions[cond.val()].length; i++) {
            var levelopt = $("<option>" + i + "</option>");
            if (i == current) {
                levelopt.attr("selected", true);
            }
            level.append(levelopt);
        }
        refresh();
    };
    
    cond.on("change", changeCond);
    level.on("change", refresh);

    element.append(cond);
    element.append(level);
    
    changeCond();
});

<IPython.core.display.Javascript object>